In [1]:
import json
import pandas as pd
from glob import glob

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.12.1 True
2.25.2
10.2
GCC 7.3


In [2]:
with open('/home/erik/Riksarkivet/Projects/lagfarter/data/file_names_lagfarter_det_2.json', 'r') as f:
    export = json.load(f)

df_lagf = pd.read_csv('/home/erik/Riksarkivet/Projects/lagfarter/data/db_files/Uttag_Lagfartsbok_2022-09-14', sep='\t', encoding='utf8', dtype=str)

In [3]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
import mmcv
from mmcv import Config

import_list = []

#img = mmcv.imread('/home/erik/Riksarkivet/Projects/lagfarter/data/det_set_import_2/51/SE_HLA1340060_CIa_4_000377.jpg')

cfg = Config.fromfile('/home/erik/Riksarkivet/Projects/lagfarter/models/configs/cascade_rcnn2.py')

checkpoint = '/home/erik/Riksarkivet/Projects/lagfarter/models/det_crcnn_iter_2_2500/latest.pth'
model = init_detector(cfg, checkpoint, device='cuda:0')

imgs = glob('/home/erik/Riksarkivet/Projects/lagfarter/data/det_set_import_2/**/*.jpg', recursive=True)

imgs = [x for x in imgs if int(x.split('/')[-2]) not in range(49,52)]

imgs.sort()

for i, inst in enumerate(export):
    
    img_p = [x for x in imgs if x.split('/')[-1] == inst['file_upload'].split('-')[1]]
    
    if len(img_p) == 0:
        continue
    else:
        img_p = img_p[0]

    try:
        arkis_nr = inst['data']['image'].split('-')[1].split('_')[1]
        series = inst['data']['image'].split('-')[1].split('_')[2]
        volume = inst['data']['image'].split('-')[1].split('_')[3]
        page_nr = str(int(inst['data']['image'].split('-')[1].split('_')[4].split('.')[0]))
        row = df_lagf[(df_lagf['ARKISNR'] == arkis_nr) & (df_lagf['ARKISSIG'] == series) & (df_lagf['ARKISVOL'] == volume) & (df_lagf['APAGE'] == page_nr)]
        
        gt = str(row.iloc[0]['SOCKEN']) + ';' + str(row.iloc[0]['FAST']) + ';' + str(row.iloc[0]['FASTNR1'])
    except:
        continue

    image_dict = dict()
    image_dict['data'] = inst['data']
    image_dict['predictions'] = list()
    image_dict['predictions'].append(dict())
    image_dict['predictions'][0]['result'] = list()
    image_dict['predictions'][0]['result'].append(dict())
    image_dict['predictions'][0]['result'][0]['value'] = dict()
    image_dict['predictions'][0]['result'][0]['value']['text'] = list()
    image_dict['predictions'][0]['result'][0]['value']['text'].append(gt)
    image_dict['predictions'][0]['result'][0]['from_name'] = 'transcription'
    image_dict['predictions'][0]['result'][0]['to_name'] = 'image'
    image_dict['predictions'][0]['result'][0]['type'] = 'textarea'
    
    img = mmcv.imread(img_p)
    result = result = inference_detector(model, img)

    for j, cat in enumerate(result):
        
        id_pred = 'result' + str(j)
        category = ''
        
        if j == 0:
            category = 'Pagenr'
        elif j == 1:
            category = 'Property'
        elif j == 2:
            category = 'Propnr'
        elif j == 3:
            category = 'Socken'

        if len(cat) == 0:
            continue
        else:
            max_cat = max(cat, key=lambda x: x[4])
            
            x = float(max_cat[0] / img.shape[1] * 100)
            y = float(max_cat[1] / img.shape[0] * 100)

            width = float((max_cat[2] - max_cat[0]) / img.shape[1] * 100)
            height = float((max_cat[3] - max_cat[1]) / img.shape[0] * 100)
            
            #image_dict['predictions'].append(dict())
            #image_dict['predictions'][-1]['model_version'] = 'one'
            #image_dict['predictions'][-1]['score'] = float(max_cat[4])
            #image_dict['predictions'][-1]['result'] = list()
            image_dict['predictions'][0]['result'].append(dict())
            image_dict['predictions'][0]['result'][-1]['id'] = id_pred
            image_dict['predictions'][0]['result'][-1]['type'] = 'rectanglelabels'
            image_dict['predictions'][0]['result'][-1]['from_name'] = 'label'
            image_dict['predictions'][0]['result'][-1]['to_name'] = 'image'
            image_dict['predictions'][0]['result'][-1]['original_width'] = float(img.shape[1])
            image_dict['predictions'][0]['result'][-1]['original_height'] = float(img.shape[0])
            image_dict['predictions'][0]['result'][-1]['image_rotation'] = 0
            image_dict['predictions'][0]['result'][-1]['value'] = dict()
            image_dict['predictions'][0]['result'][-1]['value']['rotation'] = 0
            image_dict['predictions'][0]['result'][-1]['value']['x'] = x
            image_dict['predictions'][0]['result'][-1]['value']['y'] = y
            image_dict['predictions'][0]['result'][-1]['value']['width'] = width
            image_dict['predictions'][0]['result'][-1]['value']['height'] = height
            image_dict['predictions'][0]['result'][-1]['value']['rectanglelabels'] = [category]

    import_list.append(image_dict)
    

load checkpoint from local path: /home/erik/Riksarkivet/Projects/lagfarter/models/det_crcnn_iter_2_2500/latest.pth


/home/erik/Riksarkivet/Projects/lagfarter/mmdetection/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)


In [ ]:
import_list = []

for i, inst in enumerate(export):
    

    arkis_nr = inst['data']['image'].split('-')[1].split('_')[1]
    series = inst['data']['image'].split('-')[1].split('_')[2]
    volume = inst['data']['image'].split('-')[1].split('_')[3]
    page_nr = str(int(inst['data']['image'].split('-')[1].split('_')[4].split('.')[0]))
    row = df_lagf[(df_lagf['ARKISNR'] == arkis_nr) & (df_lagf['ARKISSIG'] == series) & (df_lagf['ARKISVOL'] == volume) & (df_lagf['APAGE'] == page_nr)]
    #print(i)
    try:
        gt = str(row.iloc[0]['SOCKEN']) + ';' + str(row.iloc[0]['FAST']) + ';' + str(row.iloc[0]['FASTNR1'])
    except:
        print('error')
        continue

    image_dict = dict()
    image_dict['data'] = inst['data']
    image_dict['predictions'] = list()
    image_dict['predictions'].append(dict())
    image_dict['predictions'][0]['result'] = list()
    image_dict['predictions'][0]['result'].append(dict())
    image_dict['predictions'][0]['result'][0]['value'] = dict()
    image_dict['predictions'][0]['result'][0]['value']['text'] = list()
    image_dict['predictions'][0]['result'][0]['value']['text'].append(gt)
    image_dict['predictions'][0]['result'][0]['from_name'] = 'transcription'
    image_dict['predictions'][0]['result'][0]['to_name'] = 'image'
    image_dict['predictions'][0]['result'][0]['type'] = 'textarea'

    import_list.append(image_dict)

    

    


In [4]:
with open('/home/erik/Riksarkivet/Projects/lagfarter/output/ls_import_3_with_preds.json', 'w') as f:
    json.dump(import_list, f, indent=2, ensure_ascii=False)